In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [3]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [4]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [7]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x000001F8CEF884C0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x000001F8CEF884C0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x000001F8CEF884C0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [8]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{' 0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 2', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':5}),
           Policy('policy 3', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':10}),
            Policy('policy 4', **{'0_RfR 0':1,
                                 '0_RfR 1': 0,
                                '0_RfR 2': 0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 5', **{'0_RfR 0':0,
                                 '0_RfR 0':1,
                                '0_RfR 0':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 6', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
        Policy('policy 7', **{' 1_RfR 0':1,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 8', **{'1_RfR 0':0,
                                 '1_RfR 1':1,
                                '1_RfR 2':0,
                                'A.5_DikeIncrease 0':0}),
           Policy('policy 9', **{'1_RfR 0':0,
                                 '1_RfR 1':0,
                                '1_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
            Policy('policy 10', **{'2_RfR 0':1,
                                 '2_RfR 1':0,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 11', **{'2_RfR 0':0,
                                 '2_RfR 1':1,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 12', **{'2_RfR 0':0,
                                 '2_RfR 1':0,
                                '2_RfR 2':1,
                                'A.3_DikeIncrease 0':0})
            ]

In [9]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [10]:
use_pickle1 = True
if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)
else:
    #pass the policies list to EMA workbench experiment runs
    n_scenarios = 100

    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)


In [ ]:
import pandas as pd
a,b = results
df = pd.DataFrame(b)
df["policy"] = pd.DataFrame(a)["policy"]
df


In [ ]:
def q05(x):
    return x.quantile(0.05)
def q95(x):
    return x.quantile(0.95)

In [ ]:
group_df = df.groupby(by="policy")
agg_par = ['mean', 'min', 'max', q05, q95]
agg_dict = {
    'A.3 Total Costs': agg_par,
    'A.3_Expected Number of Deaths': agg_par,
    'RfR Total Costs':agg_par,
    'Expected Evacuation Costs':agg_par
}
TCA_df = group_df.agg(agg_dict)
TCA_df

In [ ]:
TCA_df.to_csv("results/TCA3_results.csv")